# 装饰器

函数装饰器用于在源码中"标记"函数,以某种方式增强函数的行为.严格来说装饰器这种形式是一种语法糖.

装饰器的特点有两个:

1. 装饰器是可调用的对象,其参数是另一个可调用对象.
2. 装饰器可能会处理被装饰的可调用对象,然后把它返回,或者将其替换成另一个可调用对象
3. 装饰器在加载模块时立即执行

装饰器的形式如下:

```python
@decorator
def call(args):
    pass
    
```

它等价于
call(args) = decorator(call(args))


本章需要的先验知识有:

+ [面向对象惯用法](/TutorialForPython/语法篇/面向对象惯用法/)
+ [函数](/语法篇/函数/)

## 实现装饰器

下面这个例子定义了一个装饰器,用来在每次调用被装饰的函数时计时,然后把经过的时间,传入的参数和调用的结果打印出来.

In [1]:
import time

def timer(func):
    def clocked(*args,**kw):
        t0 = time.perf_counter()
        result = func(*args,**kw)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        if args:
            arg = ",".join(repr(arg) for arg in args)
        if kw:
            kws = ",".join([repr(i)+"="+repr(v) for i,v in sorted(kw.items())])
            arg = arg+","+kws
        print("[used:{}s] function ".format(elapsed)+func.__name__+"("+arg+")->"+repr(result))
        return result
    return clocked

In [2]:
@timer
def snooze(seconds):
    time.sleep(seconds)

In [3]:
snooze(2)

[used:2.0047967199934646s] function snooze(2)->None


In [4]:
@timer
def factorial(n):
    """factorial"""
    return 1 if n < 2 else n*factorial(n-1)

In [5]:
factorial(5)

[used:7.310009095817804e-07s] function factorial(1)->1
[used:0.0001588229788467288s] function factorial(2)->2
[used:0.0001920460199471563s] function factorial(3)->6
[used:0.00022201001411303878s] function factorial(4)->24
[used:0.00025161399389617145s] function factorial(5)->120


120

In [6]:
factorial.__name__

'clocked'

## 包装装饰器

上面的装饰器有个缺点--遮盖了被装饰函数的`__name__` 和`__doc__` 属性.这时可以使用`functools.wraps` 装饰器把相关的属性从`func`复制到`clocked`中

In [7]:
import functools
def timer(func):
    @functools.wraps(func)
    def clocked(*args,**kw):
        t0 = time.perf_counter()
        result = func(*args,**kw)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        if args:
            arg = ",".join(repr(arg) for arg in args)
        if kw:
            kws = ",".join([repr(i)+"="+repr(v) for i,v in sorted(kw.items())])
            arg = arg+","+kws
        print("[used:{}s] function ".format(elapsed)+func.__name__+"("+arg+")->"+repr(result))
        return result
    return clocked

In [8]:
@timer
def snooze(seconds):
    time.sleep(seconds)

In [9]:
snooze(2)

[used:2.0031150709837675s] function snooze(2)->None


In [10]:
@timer
def factorial(n):
    """factorial"""
    return 1 if n < 2 else n*factorial(n-1)

In [11]:
factorial(6)

[used:6.059999577701092e-07s] function factorial(1)->1
[used:0.00015848499606363475s] function factorial(2)->2
[used:0.0001919149945024401s] function factorial(3)->6
[used:0.0002199050213675946s] function factorial(4)->24
[used:0.00024739297805354s] function factorial(5)->120
[used:0.0002766310062725097s] function factorial(6)->720


720

In [12]:
factorial.__name__

'factorial'

## 带参数的装饰器

我们修改之前的timer,希望它可以添加一个参数,用于指定秒数的精确位数.这样就需要写一个带参数的装饰器.

带参数的装饰器我们还需要再在外面套一层用来返回我们的装饰器函数.

In [13]:
import time
import functools
def timer(rd=3):
    def decorate(func):
        @functools.wraps(func)
        def clocked(*args,**kw):
            t0 = time.perf_counter()
            result = func(*args,**kw)
            rs = time.perf_counter() - t0
            elapsed = round(rs ,rd)
            name = func.__name__
            if args:
                arg = ",".join(repr(arg) for arg in args)
            if kw:
                kws = ",".join([repr(i)+"="+repr(v) for i,v in sorted(kw.items())])
                arg = arg+","+kws
            print("[used:{}s] function ".format(elapsed)+func.__name__+"("+arg+")->"+repr(result))
            return result
        return clocked
    return decorate

In [14]:
@timer()
def factorial(n):
    """factorial"""
    return 1 if n < 2 else n*factorial(n-1)

In [15]:
factorial(2)

[used:0.0s] function factorial(1)->1
[used:0.0s] function factorial(2)->2


2

In [16]:
@timer(7)
def factorial(n):
    """factorial"""
    return 1 if n < 2 else n*factorial(n-1)

In [17]:
factorial(2)

[used:7e-07s] function factorial(1)->1
[used:0.000142s] function factorial(2)->2


2

In [18]:
factorial.__name__

'factorial'

In [19]:
factorial.__doc__

'factorial'

当然了返回装饰器函数的对象只要是可执行对象就行.因此或许用类来实现看起来会更加自然一些

In [20]:
class timer:
    def __call__(self,func):
        @functools.wraps(func)
        def clocked(*args,**kw):
            t0 = time.perf_counter()
            result = func(*args,**kw)
            rs = time.perf_counter() - t0
            elapsed = round(rs ,self.rd)
            name = func.__name__
            if args:
                arg = ",".join(repr(arg) for arg in args)
            if kw:
                kws = ",".join([repr(i)+"="+repr(v) for i,v in sorted(kw.items())])
                arg = arg+","+kws
            print("[used:{}s] function ".format(elapsed)+func.__name__+"("+arg+")->"+repr(result))
            return result
        return clocked
    def __init__(self,rd=3):
        self.rd = rd

In [21]:
@timer(7)
def factorial(n):
    """factorial"""
    return 1 if n < 2 else n*factorial(n-1)

In [22]:
factorial(3)

[used:4e-07s] function factorial(1)->1
[used:8.65e-05s] function factorial(2)->2
[used:0.0001159s] function factorial(3)->6


6

In [23]:
factorial.__name__

'factorial'

但从开销角度来看,显然使用函数闭包实现带参数装饰器会比使用带`__call__`方法的类实例来的更加好,毕竟函数一旦定义,调用的开销远比实例化一个类小的多,但如果需要实现一些复杂的状态管理功能,这种开销或许也是值得的.

## 类装饰器

装饰器除了可以装饰函数,也可以装饰类,原理也差不多,参数是一个类,而返回的也是一个类,下面以之前的LineItem作为例子讲解如何定义和使用类装饰器.

### 定制描述符的类装饰器

在[特性与描述符部分]()的倒数第二个`LineItem`例子中储存属性的名称不具有描述性,即属性(如weight)的值存储在名为`_Quantity#0`的实例属性中,这样的名称不便于调试的问题.单靠描述符无法存储属性名字,因为实例化描述符时无法得知托管属性(即绑定到描述符上的类属性,例如前述示例中的weight)的名称.
可是,一旦组建好整个类,而且把描述符绑定到类属性上之后,我们就可以审查类,并为描述符设置合理的储存属性名称.`LineItem`类的`__new__`方法可以做到这一点,因此，在`__init__`方法中使用描述符时,储存属性已经设置了正确的名称,

为了解决这个问题而使用`__new__` 方法纯属白费力气--每次新建`LineItem`实例时都会运行`__new__` 方法中的逻辑,可是，一旦`LineItem`类构建好了,描述符与托管属性之间的绑定就不会变了.因此,我们要在创建类时设置储存属性的名称.


使用3.6的新接口`__set_name__`,类装饰器或元类都可以做到这一点.这边的例子使用类装饰器

In [24]:
def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
    return cls

In [25]:
import abc
class AutoStorage:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
    def __set_name__(self, owner, name):
        self.name = name
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        pass
class Quantity(Validated): 
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [26]:
@entity
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [27]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [28]:
LineItem.description.storage_name

'_NonBlank#description'

类装饰器能以较简单的方式做到元类做的事情——创建类时定制类.

但类装饰器也有个重大缺点:只对直接依附的类有效.这意味着,被装饰的类的子类可能继承也可能不继承装饰器所做的改动,具体情况视改动的方式而定.

## 标准库中的装饰器

Python内置了三个用于装饰方法的函数:`property`、`classmethod` 和`staticmethod`.这三个在面向对象惯用法部分讲.

而剩下的装饰器中

+ `functools.total_ordering`是用来装饰类的
+ `functools.lru_cache`,`functools.singledispatch`是用来装饰函数/方法的


### `functools.total_ordering`自动添加比较特殊方法

`functools.total_ordering`装饰器可以装饰一个类,只要其中有实现`__lt__`、`__le__`、`__gt__`、`__ge__`中的至少一个,它就会将其他的补全

In [29]:
from functools import total_ordering
@total_ordering
class Student:
    def __eq__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) ==
                (other.lastname.lower(), other.firstname.lower()))
    def __lt__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) <
                (other.lastname.lower(), other.firstname.lower()))
print(dir(Student))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


### 使用`functools.lru_cache(maxsize=128, typed=False)`做备忘


`functools.lru_cache` 是非常实用的装饰器,它实现了备忘（`memoization`）功能.这是一项优化技术,它把耗时的函数的结果保存起来,避免传入相同的参数时重复计算.LRU 三个字母是"Least Recently Used"的缩写,表明缓存不会无限制增长,一段时间不用的缓存条目会被扔掉.

+ `maxsize`参数指定存储多少个调用的结果.缓存满了之后,旧的结果会被扔掉,腾出空间.为了得到最佳性能,maxsize 应该设为2的幂.
+ typed 参数如果设为True,把不同参数类型得到的结果分开保存,即把通常认为相等的浮点数和整数参数(如1 和1.0)区分开.

因为`lru_cache`使用字典存储结果,而且键根据调用时传入的定位参数和关键字参数创建,所以被`lru_cache` 装饰的函数,它的所有参数都必须是可散列的.

生成第n个斐波纳契数这种慢速递归函数适合使用`lru_cache`

In [30]:
@timer(7)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [31]:
fibonacci(6)

[used:5e-07s] function fibonacci(0)->0
[used:9e-07s] function fibonacci(1)->1
[used:0.0001972s] function fibonacci(2)->1
[used:5e-07s] function fibonacci(1)->1
[used:5e-07s] function fibonacci(0)->0
[used:5e-07s] function fibonacci(1)->1
[used:6.06e-05s] function fibonacci(2)->1
[used:0.0001216s] function fibonacci(3)->2
[used:0.0003805s] function fibonacci(4)->3
[used:5e-07s] function fibonacci(1)->1
[used:4e-07s] function fibonacci(0)->0
[used:5e-07s] function fibonacci(1)->1
[used:5.92e-05s] function fibonacci(2)->1
[used:0.0001174s] function fibonacci(3)->2
[used:4e-07s] function fibonacci(0)->0
[used:6e-07s] function fibonacci(1)->1
[used:5.86e-05s] function fibonacci(2)->1
[used:4e-07s] function fibonacci(1)->1
[used:5e-07s] function fibonacci(0)->0
[used:6e-07s] function fibonacci(1)->1
[used:5.96e-05s] function fibonacci(2)->1
[used:0.0001193s] function fibonacci(3)->2
[used:0.0002366s] function fibonacci(4)->3
[used:0.0004132s] function fibonacci(5)->5
[used:0.0008534s] functi

8

浪费时间的地方很明显:`fibonacci(1)`调用了8 次,`fibonacci(2)`调用了5 次……但是,如果增加两行代码,使用lru_cache,性能会显著改善

In [32]:
from functools import lru_cache

In [33]:
@lru_cache()
@timer(7)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [34]:
fibonacci(6)

[used:4e-07s] function fibonacci(0)->0
[used:4e-07s] function fibonacci(1)->1
[used:7.52e-05s] function fibonacci(2)->1
[used:9e-07s] function fibonacci(3)->2
[used:0.0001163s] function fibonacci(4)->3
[used:6e-07s] function fibonacci(5)->5
[used:0.0001564s] function fibonacci(6)->8


8

***PS:装饰器的叠放顺序也是有讲究的,它是从下向上执行的,因此最终执行的结果是最上面一层的包装.***

### 使用`functools.singledispatch`实现单分配泛函

假设我们在开发一个调试Web应用的工具,我们想生成HTML,显示不同类型的Python对象,我们可能会编写这样的函数:

```python
import html
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
```

这个函数适用于任何Python类型,但是现在我们想做个扩展,让它使用特别的方式显示某些类型.

+ str：把内部的换行符替换为`\<br\>\n`；不使用`\<pre\>`，而是使用`\<p\>`。
+ int：以十进制和十六进制显示数字。
+ list：输出一个HTML列表，根据各个元素的类型进行格式化。


因为Python不支持重载方法或函数,所以我们不能使用不同的签名定义`htmlize`的变体,也无法使用不同的方式处理不同的数据类型.在Python中,一种常见的做法是把`htmlize`变成一个分派函数,使用一串`if/elif/elif`,调用专门的函数,如`htmlize_str`,`htmlize_int`,等等,这样不便于模块的用户扩展,还显得笨拙:时间一长，分派函数`htmlize`会变得很大,而且它与各个专门函数之间的耦合也很紧密.

`functools.singledispatch`装饰器可以把整体方案拆分成多个模块,甚至可以为你无法修改的类提供专门的函数.使用`@singledispatch` 装饰的普通函数会变成泛函数(`generic function`):根据第一个参数的类型,以不同方式执行相同操作的一组函数.

In [35]:
from functools import singledispatch
from collections import abc
import numbers
import html
@singledispatch 
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
@htmlize.register(str) 
def _(text): 
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)
@htmlize.register(numbers.Integral) 
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)
@htmlize.register(tuple) 
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [36]:
htmlize(123)

'<pre>123 (0x7b)</pre>'

In [37]:
htmlize('123')

'<p>123</p>'

In [38]:
htmlize([1,2,3])

'<ul>\n<li><pre>1 (0x1)</pre></li>\n<li><pre>2 (0x2)</pre></li>\n<li><pre>3 (0x3)</pre></li>\n</ul>'

只要可能,注册的专门函数应该处理抽象基类(如`numbers.Integral`和`abc.MutableSequence`),不要处理具体实现(如int 和list).这样,代码支持的兼容类型更广泛.例如,Python扩展可以子类化`numbers.Integral`,使用固定的位数实现`int` 类型.

`singledispatch`机制的一个显著特征是,你可以在系统的任何地方和任何模块中注册专门函数.如果后来在新的模块中定义了新的类型,可以轻松地添加一个新的专门函数来处理那个类型.此外,你还可以为不是自己编写的或者不能修改的类添加自定义函数.

`singledispatch` 是经过深思熟虑之后才添加到标准库中的,它提供的特性很多,这里无法一一说明.这个机制最好的文档是[PEP 443 — Single-dispatch generic functions](https://www.python.org/dev/peps/pep-0443/).

`@singledispatch`不是为了把Java的那种方法重载带入Python.在一个类中为同一个方法定义多个重载变体,比在一个函数中使用一长串`if/elif/elif/elif`块要更好.但是这两种方案都有缺陷,因为它们让代码单元(类或函数)承担的职责太多.`@singledispath`的优点是支持模块化扩展:各个模块可以为它支持的各个类型注册一个专门函数.